In [1]:
import pandas as pd
import numpy as np
import pycmap
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Gradients 4 - TN397 - Underway Hyperpro - Surface PAR.csv')

In [3]:
df

,time,lat,lon,surface_par
0,2021-11-19T16:32:00.000Z,31.165718,-119.755471,261.039130
1,2021-11-19T16:33:00.000Z,31.163667,-119.758933,302.072143
2,2021-11-19T16:34:00.000Z,31.161601,-119.762415,334.165926
3,2021-11-19T16:35:00.000Z,31.159584,-119.765842,300.941852
4,2021-11-19T16:36:00.000Z,31.157534,-119.769285,300.182593
...,...,...,...,...
34835,2021-12-13T21:07:00.000Z,14.906083,-152.971349,1298.340625
34836,2021-12-13T21:08:00.000Z,14.908998,-152.973812,1292.753636
34837,2021-12-13T21:09:00.000Z,14.911905,-152.976272,1292.897273
34838,2021-12-13T21:10:00.000Z,14.914838,-152.978767,1294.515938
